[View in Colaboratory](https://colab.research.google.com/github/SaffetGokcenSen/seq2seq_keras_test_colab/blob/master/seq2seq_keras_colab.ipynb)

In [33]:
!pip install numpy==1.14.5
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [1]:
from google.colab import files
uploaded = files.upload()

Saving tur.txt to tur.txt


In [0]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'tur.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [35]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 84
Max sequence length for inputs: 15
Max sequence length for outputs: 43


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [39]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
 - 19s - loss: 1.0730 - val_loss: 1.0976
Epoch 2/100
 - 18s - loss: 0.8364 - val_loss: 0.8816
Epoch 3/100
 - 17s - loss: 0.7017 - val_loss: 0.7645
Epoch 4/100
 - 18s - loss: 0.6295 - val_loss: 0.7098
Epoch 5/100
 - 18s - loss: 0.5811 - val_loss: 0.6600
Epoch 6/100
 - 17s - loss: 0.5417 - val_loss: 0.6221
Epoch 7/100
 - 18s - loss: 0.5082 - val_loss: 0.5953
Epoch 8/100
 - 18s - loss: 0.4786 - val_loss: 0.5794
Epoch 9/100
 - 18s - loss: 0.4527 - val_loss: 0.5607
Epoch 10/100
 - 18s - loss: 0.4287 - val_loss: 0.5467
Epoch 11/100
 - 18s - loss: 0.4063 - val_loss: 0.5342
Epoch 12/100
 - 18s - loss: 0.3854 - val_loss: 0.5309
Epoch 13/100
 - 18s - loss: 0.3660 - val_loss: 0.5228
Epoch 14/100
 - 18s - loss: 0.3469 - val_loss: 0.5199
Epoch 15/100
 - 18s - loss: 0.3289 - val_loss: 0.5188
Epoch 16/100
 - 18s - loss: 0.3118 - val_loss: 0.5215
Epoch 17/100
 - 18s - loss: 0.2957 - val_loss: 0.5223
Epoch 18/100
 - 18s - loss: 0.2803 - val_lo

In the above results, after epoch 25, overfitting starts since the loss of the validation set starts to increase as the loss of the training set decreases. However, the performance of the algorithm on the training set is to be observed.

In [40]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Merhaba.

-
Input sentence: Hi.
Decoded sentence: Merhaba.

-
Input sentence: Run!
Decoded sentence: Kaç!

-
Input sentence: Run!
Decoded sentence: Kaç!

-
Input sentence: Run.
Decoded sentence: Kaç!

-
Input sentence: Run.
Decoded sentence: Kaç!

-
Input sentence: Who?
Decoded sentence: Kim?

-
Input sentence: Fire!
Decoded sentence: Ateş!

-
Input sentence: Fire!
Decoded sentence: Ateş!

-
Input sentence: Help!
Decoded sentence: Yardım et!

-
Input sentence: Jump.
Decoded sentence: Defol.

-
Input sentence: Stop!
Decoded sentence: Dur!

-
Input sentence: Stop!
Decoded sentence: Dur!

-
Input sentence: Wait.
Decoded sentence: Bekleyin.

-
Input sentence: Do it.
Decoded sentence: Onu yap.

-
Input sentence: Go on.
Decoded sentence: Devam et.

-
Input sentence: Hello!
Decoded sentence: Merhaba.

-
Input sentence: Hello!
Decoded sentence: Merhaba.

-
Input sentence: Hurry!
Decoded sentence: Acele et!

-
Input sentence: I ran.
Decoded sentence: Koşt